# Module 4: Image Transformations

Welcome to Module 4! In this notebook, you'll learn how to **transform** images in various ways.

## What You'll Learn

By the end of this module, you'll be able to:
- **Resize** images while maintaining quality
- **Translate** (shift) images in any direction
- **Rotate** images around any point
- **Correct perspective** distortion (like in document scanning)
- Understand different **interpolation methods**
- Implement basic **watermarking** techniques
- Measure image quality with **PSNR**

## Real-World Applications

- 📱 **Photo resizing** for social media and web
- 📄 **Document scanning** apps (perspective correction)
- 🔄 **Image alignment** for stitching panoramas
- 🔐 **Digital watermarking** for copyright protection
- 🎨 **Photo editing** tools

---

## Setup

---

**⏱️ Estimated Time**: 90-120 minutes  
**📚 Level**: Intermediate  
**📋 Prerequisites**: Completed notebooks 00-03

---

In [ ]:
# Import required libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

# For inline plotting in Jupyter
%matplotlib inline

# Set default figure size
plt.rcParams["figure.figsize"] = (12, 8)

print("✓ Libraries imported successfully!")
print(f"OpenCV version: {cv.__version__}")

In [ ]:
# Helper function to display images side by side
def show_images(images, titles, figsize=(15, 5), cmap=None):
    """
    Display multiple images in a row.

    Parameters:
    - images: list of images to display
    - titles: list of titles for each image
    - figsize: tuple for figure size
    - cmap: colormap (use 'gray' for grayscale images)
    """
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)

    if n == 1:
        axes = [axes]

    for i, (img, title) in enumerate(zip(images, titles)):
        if len(img.shape) == 3 and img.shape[2] == 3:
            # Convert BGR to RGB for color images
            axes[i].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        else:
            # Grayscale image
            axes[i].imshow(img, cmap="gray" if cmap is None else cmap)
        axes[i].set_title(title)
        axes[i].axis("off")

    plt.tight_layout()
    plt.show()


print("✓ Helper function defined!")

In [ ]:
# Create a sample image with text and shapes for testing
img = np.ones((400, 600, 3), dtype=np.uint8) * 255  # White background

# Draw some shapes and text
cv.rectangle(img, (50, 50), (250, 250), (255, 0, 0), -1)  # Blue rectangle
cv.circle(img, (450, 150), 80, (0, 255, 0), -1)  # Green circle
cv.ellipse(img, (300, 300), (100, 50), 45, 0, 360, (0, 0, 255), -1)  # Red ellipse
cv.putText(img, "TRANSFORM", (150, 380), cv.FONT_HERSHEY_DUPLEX, 1.5, (0, 0, 0), 3)

# Display
plt.figure(figsize=(8, 6))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.title("Sample Image for Transformations")
plt.axis("off")
plt.show()

print(f"Sample image created: {img.shape[1]}x{img.shape[0]} pixels")

---

## Part 1: Scaling (Resizing Images)

**Scaling** changes the size of an image. You can make it larger (upscaling) or smaller (downscaling).

### Key Function: `cv.resize()`

```python
resized = cv.resize(img, (new_width, new_height), interpolation=method)
```

### Interpolation Methods

When resizing, pixels need to be **interpolated** (estimated). Different methods give different quality:

| Method | Description | Best For |
|--------|-------------|----------|
| `INTER_NEAREST` | Nearest neighbor (fastest) | Quick previews, pixel art |
| `INTER_LINEAR` | Bilinear interpolation | General shrinking |
| `INTER_CUBIC` | Bicubic interpolation | High-quality resizing |
| `INTER_LANCZOS4` | Lanczos (best quality) | Professional photography |
| `INTER_AREA` | Pixel area relation | Shrinking images |

**Rule of thumb**:
- **Shrinking**: Use `INTER_AREA` or `INTER_LINEAR`
- **Enlarging**: Use `INTER_CUBIC` or `INTER_LANCZOS4`

In [ ]:
# Example 1: Resize to specific dimensions

# Shrink to 300x200 pixels
small = cv.resize(img, (300, 200), interpolation=cv.INTER_AREA)

# Enlarge to 900x600 pixels
large = cv.resize(img, (900, 600), interpolation=cv.INTER_CUBIC)

show_images(
    [small, img, large],
    [
        f"Small ({small.shape[1]}x{small.shape[0]})",
        f"Original ({img.shape[1]}x{img.shape[0]})",
        f"Large ({large.shape[1]}x{large.shape[0]})",
    ],
    figsize=(18, 5),
)

In [ ]:
# Example 2: Resize by scaling factor (maintaining aspect ratio)

scale_percent = 50  # Resize to 50% of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)

resized_50 = cv.resize(img, (width, height), interpolation=cv.INTER_AREA)

print(f"Original: {img.shape[1]}x{img.shape[0]}")
print(f"50% scale: {resized_50.shape[1]}x{resized_50.shape[0]}")

show_images([img, resized_50], ["Original", "50% Scale"])

In [ ]:
# Example 3: Compare interpolation methods

# Create a small image for upscaling test
small_test = cv.resize(img, (150, 100), interpolation=cv.INTER_AREA)

# Upscale using different methods
nearest = cv.resize(small_test, (600, 400), interpolation=cv.INTER_NEAREST)
linear = cv.resize(small_test, (600, 400), interpolation=cv.INTER_LINEAR)
cubic = cv.resize(small_test, (600, 400), interpolation=cv.INTER_CUBIC)
lanczos = cv.resize(small_test, (600, 400), interpolation=cv.INTER_LANCZOS4)

# Display comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
methods = [nearest, linear, cubic, lanczos]
titles = [
    "INTER_NEAREST (Blocky)",
    "INTER_LINEAR (Good)",
    "INTER_CUBIC (Better)",
    "INTER_LANCZOS4 (Best)",
]

for i, (method, title) in enumerate(zip(methods, titles)):
    row, col = i // 2, i % 2
    axes[row, col].imshow(cv.cvtColor(method, cv.COLOR_BGR2RGB))
    axes[row, col].set_title(title, fontsize=14)
    axes[row, col].axis("off")

plt.tight_layout()
plt.show()

print("Notice: NEAREST is blocky, while LANCZOS4 is smoothest!")

---

## Part 2: Translation (Shifting Images)

**Translation** means moving an image horizontally and/or vertically without rotating or resizing.

### How It Works

1. Create a **translation matrix** (2x3 matrix)
2. Use `cv.warpAffine()` to apply the transformation

### Translation Matrix

```
M = | 1  0  tx |
    | 0  1  ty |
```

Where:
- `tx` = horizontal shift (positive = right, negative = left)
- `ty` = vertical shift (positive = down, negative = up)

In [ ]:
# Example: Shift image right and down

tx = 100  # Move 100 pixels right
ty = 50  # Move 50 pixels down

# Create translation matrix
M_translate = np.float32([[1, 0, tx], [0, 1, ty]])

# Apply translation
rows, cols = img.shape[:2]
translated = cv.warpAffine(img, M_translate, (cols, rows))

show_images([img, translated], ["Original", f"Translated ({tx}px right, {ty}px down)"])

print(f"Translation matrix:\n{M_translate}")

In [ ]:
# Example: Shift in different directions

# Right
M_right = np.float32([[1, 0, 100], [0, 1, 0]])
shift_right = cv.warpAffine(img, M_right, (cols, rows))

# Left
M_left = np.float32([[1, 0, -100], [0, 1, 0]])
shift_left = cv.warpAffine(img, M_left, (cols, rows))

# Up
M_up = np.float32([[1, 0, 0], [0, 1, -50]])
shift_up = cv.warpAffine(img, M_up, (cols, rows))

# Down
M_down = np.float32([[1, 0, 0], [0, 1, 50]])
shift_down = cv.warpAffine(img, M_down, (cols, rows))

# Display
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
shifts = [shift_right, shift_left, shift_up, shift_down]
titles = ["Right (+100, 0)", "Left (-100, 0)", "Up (0, -50)", "Down (0, +50)"]

for i, (shifted, title) in enumerate(zip(shifts, titles)):
    row, col = i // 2, i % 2
    axes[row, col].imshow(cv.cvtColor(shifted, cv.COLOR_BGR2RGB))
    axes[row, col].set_title(title, fontsize=14)
    axes[row, col].axis("off")

plt.tight_layout()
plt.show()

---

## Part 3: Rotation

**Rotation** spins an image around a center point by a specified angle.

### Key Functions

```python
# Get rotation matrix
M = cv.getRotationMatrix2D(center, angle, scale)

# Apply rotation
rotated = cv.warpAffine(img, M, (width, height))
```

Parameters:
- `center`: Tuple (x, y) - point to rotate around
- `angle`: Rotation angle in **degrees** (positive = counter-clockwise)
- `scale`: Scaling factor (1.0 = no scaling)

In [ ]:
# Example 1: Rotate around image center

# Get image center
center_x = cols // 2
center_y = rows // 2
center = (center_x, center_y)

# Rotate 45 degrees counter-clockwise
angle = 45
scale = 1.0

# Get rotation matrix
M_rotate = cv.getRotationMatrix2D(center, angle, scale)

# Apply rotation
rotated = cv.warpAffine(img, M_rotate, (cols, rows))

show_images([img, rotated], ["Original", f"Rotated {angle}°"])

print(f"Rotation matrix:\n{M_rotate}")

In [ ]:
# Example 2: Rotate at different angles

angles = [0, 45, 90, 135, 180, 270]
rotated_images = []

for angle in angles:
    M = cv.getRotationMatrix2D(center, angle, 1.0)
    rotated_img = cv.warpAffine(img, M, (cols, rows))
    rotated_images.append(rotated_img)

# Display
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (rotated_img, angle) in enumerate(zip(rotated_images, angles)):
    axes[i].imshow(cv.cvtColor(rotated_img, cv.COLOR_BGR2RGB))
    axes[i].set_title(f"{angle}°", fontsize=14)
    axes[i].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Example 3: Rotate without cropping (expand canvas)

angle = 45

# Calculate new bounding dimensions
radians = np.deg2rad(angle)
sin = abs(np.sin(radians))
cos = abs(np.cos(radians))

new_width = int((rows * sin) + (cols * cos))
new_height = int((rows * cos) + (cols * sin))

# Get rotation matrix
M = cv.getRotationMatrix2D(center, angle, 1.0)

# Adjust translation to center the image
M[0, 2] += (new_width / 2) - center_x
M[1, 2] += (new_height / 2) - center_y

# Apply rotation with expanded canvas
rotated_full = cv.warpAffine(
    img, M, (new_width, new_height), borderMode=cv.BORDER_CONSTANT, borderValue=(255, 255, 255)
)

# Compare cropped vs full
M_crop = cv.getRotationMatrix2D(center, angle, 1.0)
rotated_crop = cv.warpAffine(img, M_crop, (cols, rows))

show_images(
    [rotated_crop, rotated_full],
    ["Rotated (Cropped)", "Rotated (Full - No Cropping)"],
    figsize=(16, 6),
)

---

## Part 4: Perspective Transformation

**Perspective transformation** corrects the perspective distortion of images (like photos of documents taken at an angle).

### How It Works

1. Define **4 source points** (corners in original image)
2. Define **4 destination points** (where corners should be)
3. Get transformation matrix with `cv.getPerspectiveTransform()`
4. Apply with `cv.warpPerspective()`

**Use Cases**:
- Document scanning apps
- Correcting photos of whiteboards
- Bird's eye view transformation
- License plate normalization

In [ ]:
# Example: Simulate document scanning

# Create a "document" image
doc = np.ones((400, 300, 3), dtype=np.uint8) * 255
cv.rectangle(doc, (20, 20), (280, 380), (0, 0, 0), 3)
cv.putText(doc, "DOCUMENT", (50, 100), cv.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 2)
cv.putText(doc, "Line 1", (40, 180), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
cv.putText(doc, "Line 2", (40, 230), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
cv.putText(doc, "Line 3", (40, 280), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)

# Apply perspective distortion (simulate angled photo)
src_pts = np.float32([[0, 0], [300, 0], [300, 400], [0, 400]])
dst_pts_skewed = np.float32([[50, 30], [280, 10], [320, 380], [20, 390]])

M_skew = cv.getPerspectiveTransform(src_pts, dst_pts_skewed)
skewed = cv.warpPerspective(
    doc, M_skew, (400, 400), borderMode=cv.BORDER_CONSTANT, borderValue=(200, 200, 200)
)

# Now correct the perspective
# Source points: corners of skewed document
src_correction = np.float32([[50, 30], [280, 10], [320, 380], [20, 390]])

# Destination points: where we want corners to be (rectangular)
dst_correction = np.float32([[0, 0], [300, 0], [300, 400], [0, 400]])

# Get perspective transform matrix
M_correct = cv.getPerspectiveTransform(src_correction, dst_correction)

# Apply correction
corrected = cv.warpPerspective(skewed, M_correct, (300, 400))

# Display
show_images(
    [doc, skewed, corrected],
    ["Original Document", "Skewed (Angled Photo)", "Corrected (Scanned)"],
    figsize=(18, 5),
)

print("✓ Perspective transformation applied!")
print("This is how document scanning apps work!")

---

## Part 5: Digital Watermarking - LSB Method

**Digital watermarking** embeds hidden information (like copyright data) into images.

### LSB (Least Significant Bit) Method

**Concept**: Modify the least significant bit of pixel values to hide information.

**Why it works**:
- Changing the last bit of a pixel (e.g., 255 → 254) is imperceptible to human eyes
- Can hide entire images or text

**Example**:
```
Pixel value: 10110110 (182)
Hidden bit:  1
New value:   10110111 (183)  ← Difference of 1 is invisible!
```

### Advantages and Disadvantages

**Advantages**:
- ✅ Simple to implement
- ✅ Imperceptible to human eye
- ✅ High capacity (can hide lots of data)

**Disadvantages**:
- ❌ Not robust (destroyed by compression, resizing, rotation)
- ❌ Easy to remove if someone knows about it

In [ ]:
# Example: Simple LSB watermarking


def embed_watermark_lsb(host_img, watermark_img):
    """
    Embed watermark into host image using LSB method.

    Parameters:
    - host_img: Original image (will be modified)
    - watermark_img: Watermark to hide (must be smaller or equal size)

    Returns:
    - watermarked: Image with embedded watermark
    """
    # Ensure images are same size (resize watermark if needed)
    if watermark_img.shape != host_img.shape:
        watermark_img = cv.resize(watermark_img, (host_img.shape[1], host_img.shape[0]))

    # Convert to binary (threshold)
    if len(watermark_img.shape) == 3:
        watermark_gray = cv.cvtColor(watermark_img, cv.COLOR_BGR2GRAY)
    else:
        watermark_gray = watermark_img

    _, watermark_binary = cv.threshold(watermark_gray, 127, 1, cv.THRESH_BINARY)

    # Create watermarked image
    watermarked = host_img.copy()

    # Embed watermark in blue channel (LSB)
    # Clear LSB, then set it to watermark bit
    watermarked[:, :, 0] = (watermarked[:, :, 0] & 0xFE) | watermark_binary

    return watermarked


def extract_watermark_lsb(watermarked_img):
    """
    Extract watermark from watermarked image.

    Parameters:
    - watermarked_img: Image with embedded watermark

    Returns:
    - watermark: Extracted watermark
    """
    # Extract LSB from blue channel
    watermark = (watermarked_img[:, :, 0] & 1) * 255

    return watermark.astype(np.uint8)


print("✓ Watermarking functions defined!")

In [ ]:
# Create a watermark (simple text)
watermark = np.zeros((400, 600), dtype=np.uint8)
cv.putText(watermark, "COPYRIGHT", (150, 220), cv.FONT_HERSHEY_DUPLEX, 2, 255, 4)

# Embed watermark into our sample image
watermarked = embed_watermark_lsb(img, watermark)

# Extract watermark
extracted = extract_watermark_lsb(watermarked)

# Display
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
axes[0, 0].set_title("Original Image", fontsize=14)
axes[0, 0].axis("off")

axes[0, 1].imshow(watermark, cmap="gray")
axes[0, 1].set_title("Watermark to Hide", fontsize=14)
axes[0, 1].axis("off")

axes[1, 0].imshow(cv.cvtColor(watermarked, cv.COLOR_BGR2RGB))
axes[1, 0].set_title("Watermarked Image (Looks Same!)", fontsize=14)
axes[1, 0].axis("off")

axes[1, 1].imshow(extracted, cmap="gray")
axes[1, 1].set_title("Extracted Watermark", fontsize=14)
axes[1, 1].axis("off")

plt.tight_layout()
plt.show()

print("Notice: The watermarked image looks identical to the original!")
print("But the watermark is hidden in the LSB and can be extracted.")

---

## Part 6: Measuring Image Quality - PSNR

**PSNR** (Peak Signal-to-Noise Ratio) measures how much an image has been degraded.

### Formula

$$
\text{MSE} = \frac{1}{m \times n} \sum_{i=0}^{m-1} \sum_{j=0}^{n-1} [I(i,j) - K(i,j)]^2
$$

$$
\text{PSNR} = 10 \cdot \log_{10}\left(\frac{MAX^2}{MSE}\right)
$$

Where:
- `MSE` = Mean Squared Error
- `MAX` = Maximum pixel value (255 for 8-bit images)

### Interpreting PSNR

| PSNR (dB) | Quality |
|-----------|----------|
| > 40 | Excellent (imperceptible difference) |
| 30-40 | Good (small differences) |
| 20-30 | Fair (noticeable differences) |
| < 20 | Poor (significant degradation) |

**Higher PSNR = Better quality (less degradation)**

In [ ]:
def calculate_psnr(original, modified):
    """
    Calculate PSNR between two images.

    Parameters:
    - original: Original image
    - modified: Modified image

    Returns:
    - psnr: PSNR value in dB
    """
    # Ensure images are same size
    if original.shape != modified.shape:
        modified = cv.resize(modified, (original.shape[1], original.shape[0]))

    # Calculate MSE
    mse = np.mean((original.astype(float) - modified.astype(float)) ** 2)

    # Avoid division by zero
    if mse == 0:
        return float("inf")  # Images are identical

    # Calculate PSNR
    max_pixel = 255.0
    psnr = 10 * np.log10((max_pixel**2) / mse)

    return psnr


print("✓ PSNR function defined!")

In [ ]:
# Test PSNR with different quality images

# 1. Watermarked image (should have very high PSNR)
psnr_watermark = calculate_psnr(img, watermarked)

# 2. Slightly compressed (add small noise)
noise = np.random.normal(0, 5, img.shape).astype(np.int16)
slightly_degraded = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
psnr_slight = calculate_psnr(img, slightly_degraded)

# 3. Heavily compressed (add more noise)
noise_heavy = np.random.normal(0, 20, img.shape).astype(np.int16)
heavily_degraded = np.clip(img.astype(np.int16) + noise_heavy, 0, 255).astype(np.uint8)
psnr_heavy = calculate_psnr(img, heavily_degraded)

# Display results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
axes[0, 0].set_title("Original Image", fontsize=14)
axes[0, 0].axis("off")

axes[0, 1].imshow(cv.cvtColor(watermarked, cv.COLOR_BGR2RGB))
axes[0, 1].set_title(f"Watermarked\nPSNR: {psnr_watermark:.2f} dB (Excellent)", fontsize=14)
axes[0, 1].axis("off")

axes[1, 0].imshow(cv.cvtColor(slightly_degraded, cv.COLOR_BGR2RGB))
axes[1, 0].set_title(f"Slightly Degraded\nPSNR: {psnr_slight:.2f} dB (Good)", fontsize=14)
axes[1, 0].axis("off")

axes[1, 1].imshow(cv.cvtColor(heavily_degraded, cv.COLOR_BGR2RGB))
axes[1, 1].set_title(f"Heavily Degraded\nPSNR: {psnr_heavy:.2f} dB (Fair)", fontsize=14)
axes[1, 1].axis("off")

plt.tight_layout()
plt.show()

print("\nPSNR Summary:")
print(f"Watermarked:        {psnr_watermark:.2f} dB (Imperceptible)")
print(f"Slightly Degraded:  {psnr_slight:.2f} dB")
print(f"Heavily Degraded:   {psnr_heavy:.2f} dB")
print("\nHigher PSNR = Better quality!")

---

## Part 7: Practical Exercises

Now it's your turn to practice! Try these exercises to reinforce your learning.

### Exercise 1: Photo Resizer Tool

Create a tool that resizes an image for different social media platforms:
- Instagram Post: 1080x1080
- Instagram Story: 1080x1920
- Twitter Header: 1500x500

Compare the quality using different interpolation methods.

In [ ]:
# Exercise 1: Your code here

# TODO: Resize 'img' to Instagram Post size (1080x1080)
# TODO: Try INTER_LINEAR, INTER_CUBIC, and INTER_LANCZOS4
# TODO: Display all three and compare quality

# Hint: Use cv.resize() with different interpolation methods

# Your solution:
instagram_linear = cv.resize(img, (1080, 1080), interpolation=cv.INTER_LINEAR)
instagram_cubic = cv.resize(img, (1080, 1080), interpolation=cv.INTER_CUBIC)
instagram_lanczos = cv.resize(img, (1080, 1080), interpolation=cv.INTER_LANCZOS4)

show_images(
    [instagram_linear, instagram_cubic, instagram_lanczos],
    ["LINEAR", "CUBIC", "LANCZOS4"],
    figsize=(18, 6),
)

print("✓ Exercise 1 complete!")

### Exercise 2: Create a 360° Rotation Animation

Create a list of images rotated from 0° to 360° in 30° increments.
Display them in a grid to show the rotation sequence.

In [ ]:
# Exercise 2: Your code here

# TODO: Create rotations from 0° to 360° in 30° steps
# TODO: Display in a 3x4 grid

# Your solution:
angles = range(0, 360, 30)
center = (img.shape[1] // 2, img.shape[0] // 2)
rotation_sequence = []

for angle in angles:
    M = cv.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv.warpAffine(img, M, (img.shape[1], img.shape[0]))
    rotation_sequence.append(rotated)

# Display in 3x4 grid
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for i, (rotated, angle) in enumerate(zip(rotation_sequence, angles)):
    axes[i].imshow(cv.cvtColor(rotated, cv.COLOR_BGR2RGB))
    axes[i].set_title(f"{angle}°")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

print("✓ Exercise 2 complete!")

### Exercise 3: Test Watermark Robustness

Test how robust the LSB watermark is by:
1. Adding noise to the watermarked image
2. Resizing it
3. Rotating it slightly

Try to extract the watermark after each modification and see what happens!

In [ ]:
# Exercise 3: Your code here

# TODO: Add small noise to watermarked image
# TODO: Try to extract watermark from noisy version
# TODO: Compare with original extracted watermark

# Your solution:
# Add noise
noise = np.random.normal(0, 3, watermarked.shape).astype(np.int16)
watermarked_noisy = np.clip(watermarked.astype(np.int16) + noise, 0, 255).astype(np.uint8)

# Resize (50% then back to original size)
small = cv.resize(watermarked, (300, 200), interpolation=cv.INTER_AREA)
watermarked_resized = cv.resize(small, (600, 400), interpolation=cv.INTER_CUBIC)

# Extract watermarks
extracted_noisy = extract_watermark_lsb(watermarked_noisy)
extracted_resized = extract_watermark_lsb(watermarked_resized)

# Display
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].imshow(extracted, cmap="gray")
axes[0, 0].set_title("Original Extracted Watermark")
axes[0, 0].axis("off")

axes[0, 1].imshow(cv.cvtColor(watermarked_noisy, cv.COLOR_BGR2RGB))
axes[0, 1].set_title("Watermarked + Noise")
axes[0, 1].axis("off")

axes[1, 0].imshow(extracted_noisy, cmap="gray")
axes[1, 0].set_title("Extracted from Noisy (Degraded!)")
axes[1, 0].axis("off")

axes[1, 1].imshow(extracted_resized, cmap="gray")
axes[1, 1].set_title("Extracted from Resized (Destroyed!)")
axes[1, 1].axis("off")

plt.tight_layout()
plt.show()

print("✓ Exercise 3 complete!")
print("Notice: LSB watermarking is NOT robust to modifications!")

---

## Summary

Congratulations! You've learned about **Image Transformations**! 🎉

### Key Takeaways

1. **Scaling (Resizing)**:
   - Use `cv.resize()` with appropriate interpolation
   - INTER_AREA for shrinking, INTER_CUBIC/LANCZOS4 for enlarging

2. **Translation (Shifting)**:
   - Create translation matrix with `np.float32([[1, 0, tx], [0, 1, ty]])`
   - Apply with `cv.warpAffine()`

3. **Rotation**:
   - Get rotation matrix with `cv.getRotationMatrix2D(center, angle, scale)`
   - Apply with `cv.warpAffine()`
   - Can rotate around any point

4. **Perspective Transformation**:
   - Define 4 source and destination points
   - Use `cv.getPerspectiveTransform()` and `cv.warpPerspective()`
   - Perfect for document scanning and correction

5. **Digital Watermarking**:
   - LSB method hides data in least significant bits
   - Imperceptible but not robust to modifications
   - Use PSNR to measure quality (higher = better)

### Real-World Applications You Can Build

- 📱 Photo resizer for social media
- 📄 Document scanner app
- 🎨 Photo editor with rotation/flip tools
- 🔐 Copyright protection system
- 🗺️ Image alignment for panoramas

### Next Steps

In the next module, you'll learn about **Morphological Operations** - powerful techniques for processing binary images and detecting shapes!

---

**Great work! You're making excellent progress!** 🚀